## Reading large QTl files based off QTLStudy structure

In [ ]:
using DLMReader
using Chain
using InMemoryDatasets
import Base.Threads.@threads
using StatsBase

export QTLStudy, GWAS, QTLStudy_from_pattern, GenVarInfo
include("../src/inputs.jl")
include("../src/mrStudyCis.jl")

In [ ]:
GWAS_file_path = "/mnt/r6_data/GWAS_MRsuite_hg38/input_MRsuite_BMI_Yengo_hg38"
QTL_prots_path = "/home/samat33/CrickData_samat33/Decode_mapping_suite"
QTL_prots_tss_file = "/mnt/r6_data/Patrick/Input_Decode/decode_list_hg38"

tss_df = filereader(QTL_prots_tss_file, delimiter = '_', header=[:trait, :chr, :tss], eolwarn = false)

dropmissing!(tss_df)

Decode_pat = [TRAIT_NAME, "/", TRAIT_NAME, "_chromosome_", CHR, ".tsv"]
Decode_columns = Dict(1 => "id", 2 => CHR, 3 => POS, 4 => A_EFFECT, 5 => A_OTHER, 6 => BETA, 7 => SE, 8 => PVAL)
sep = '\t'

Decode = QTLStudy_from_pattern(QTL_prots_path,Decode_pat, tss_df.trait, tss_df.chr, tss_df.tss, Decode_columns, sep, true)

display(Decode.path_v)
display(Decode.traits_for_each_path)

Read each file : keep only relevant cols and smoosh it all together

In [ ]:
Decode[1]

In [ ]:
verify_and_simplify_columns(Decode)

In [ ]:
types, header = make_types_and_headers(Decode)

Next : test speed lecture pour quelques fichiers seulement manuelement

In [ ]:
file = Decode[1]
d = filereader(file.path, delimiter = file.separator, header = header, types = types, skipto=2, makeunique=true, eolwarn=false, threads=false)[:,collect(keys(file.columns))]

In [ ]:
window = 500000
p_tresh = 5.0e-3
in_window(s::SubArray) = (s[1] == ref_dict[s[3]][1] && abs(s[2]-ref_dict[s[3]][2])≤window && s[4]<p_tresh)
ref_dict = Dict(zip(Decode.trait_v, zip(Decode.chr_v, Decode.tss_v)))

In [ ]:
data_vect = Vector{Dataset}(undef, length(Decode));

In [ ]:
using Printf
using BenchmarkTools
using Folds
using BangBang

In [ ]:

@time @threads for i in 1:lastindex(Decode)
    @printf "\r%5i" i
    file = Decode[i]
    d = filereader(file.path, delimiter = file.separator, header = header, types = types, skipto=2, makeunique=true, eolwarn=false, threads = false)[:,collect(keys(file.columns))]
    d.trait = repeat([file.trait_name], nrow(d))
    filter!(d, [:chr, :pos, :trait, :pval], type = in_window, threads = false)
    data_vect[i] = d
end

**Mem taken $\sim$ 315 Gb. look workspace for temporary allocations + other optimisation**

In [ ]:
df_final = Folds.reduce(vcat, data_vect, init = Dataset())
data_vect = 0
GC.gc()

In [ ]:
groupby!(df_final, [:chr, :pos])

In [ ]:
df_final

In [ ]:
filewriter("/mnt/r6_data2/Decode_cis_filtered/Decode_cis_filteres.csv", df_final)

takes 151 Mb. Read this file alone to compare!!!

**Solution multithread** : 
```
si nb_files > nb_threads : @threads for_loop
else : for_loop with threads = true in filereader and filter
```

In [ ]:
dada_retreived = filereader("/mnt/r6_data2/Decode_cis_filtered/Decode_cis_filteres.csv")

In [ ]:
groupby!(dada_retreived, [:chr, :pos])